In [1]:
import re
import numpy as np
import itertools

In [2]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrix']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [3]:
filename = r"Xpress Data Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrix'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [4]:
Allocations

array([45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56,
       83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
       90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
       21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
       54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
        1, 33, 59, 42, 52,  0,  0,  0,  0,  0,  0])

In [5]:
def q1_function(allocation_vector, DistancesMatrix):
    def find_closest_product(current_position, products, distances):
        closest_product = None
        min_distance = float('inf')
        for product in products:
            if distances[current_position][product] <= min_distance:
                closest_product = product
                min_distance = distances[current_position][product]
        return closest_product

    def generate_order_lists(order):
        ''' 
        Takes a list with tuple elements and returns a list of lists 
        with all possible combinations of individual elements individual elements.
        '''
        order_lists = []
        tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
        for combination in itertools.product(*[order[i] for i in tuple_indices]):
            new_order = order.copy()
            for i, index in enumerate(tuple_indices):
                new_order[index] = combination[i]
            order_lists.append(new_order)
        return order_lists

    def greedy_order_route(order, distances):
        ''' 
        Uses a greedy method of calculating the minimum distance. 

        Function has been split into two if statements to consider
        cases of orders where products are contained on more than one shelf.

        If products are contained on more than one shelf, the function constructs 
        a route with all possible shelf combinations and chooses the one with 
        the shortest distance.

        Returns a list containing the route and the total distance for the order.
        '''

        # If all products in the order are contained on one shelf only
        if not any(isinstance(product, tuple) for product in order):
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
            visited.append(0)
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distance_final = OrderDistance
            visited_final = visited

        # If one or more products in the order are contained on more than one shelf
        elif any(isinstance(product, tuple) for product in order):
            order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
            order_routes = []   # initialise a list of routes for all combinations                                 
            order_distances = [] # initialise a list of distances for all combinations
            
            # loop over all combinations
            for order in order_combinations:
                visited = [0]
                current_position = 0  
                for k in range(len(order)):
                    closest_product = find_closest_product(current_position, order, distances)
                    visited.append(closest_product)
                    order.remove(closest_product)
                    current_position = closest_product
                visited.append(0)
                order_routes.append(visited) # add the route for the combination to the list of routes
                OrderDistance = 0
                for i in range(len(visited) - 1):
                    OrderDistance += distances[visited[i]][visited[i+1]]
                order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
            
            # select the order with the shortest distance among the combinations
            min_idx = order_distances.index(min(order_distances))
            visited_final = order_routes[min_idx] 
            order_distance_final = order_distances[min_idx]
                
        return visited_final, order_distance_final # return order route and distance
    
    def convert_orders_to_shelf_indices(allocations):
        ''' 
        This function takes the allocation vector and returns an 
        order matrix with shelf indices instead of product indices.
        '''
        product_to_shelf = {}
        for shelf_index, product in enumerate(allocations):
            if product != 0:  # Check if the element is not zero
                if product not in product_to_shelf:
                    product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
                else:
                    # If the product already exists in the dictionary, append the new shelf index to the list
                    product_to_shelf[product].append(shelf_index + 1)

        # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
        product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

        OrdersByShelf = []
        for order in Orders:
            order_shelf_indices = []
            for product_index in order:
                if product_index in product_to_shelf_tuples:
                    shelf_indices = product_to_shelf_tuples[product_index]
                    order_shelf_indices.append(shelf_indices)
                else:
                    order_shelf_indices.append(0)  # Product not found in allocation matrix
            OrdersByShelf.append(order_shelf_indices)

        return OrdersByShelf
    

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, DistancesMatrix)
        routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)
    
    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes

In [6]:
### TEST CASE ON CURRENT ALLOCATION ###
TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, routes = q1_function(Allocations)

TypeError: q1_function() missing 1 required positional argument: 'DistancesMatrix'

#### Obtaining total distances for different allocations

In [7]:
print(f"Distance for the current allocation: {q1_function(Allocations)[0]} metres.")
print(f"Distance for all shelves filled: {q1_function(allocations_full)[0]} metres.")
print(f"Distance for some repeated products and some empty shelves: {q1_function(allocations_mid_full)[0]} metres.")

TypeError: q1_function() missing 1 required positional argument: 'DistancesMatrix'

In [8]:
number_to_fill = 1000

# Number of rows and columns to add
num_rows_to_add = 96
num_cols_to_add = 96


# Create a new row filled with the specified number
new_row = np.full((1, FullDistanceMatrix.shape[1]), number_to_fill)

# Repeat the new row to match the number of rows to add
new_rows = np.repeat(new_row, num_rows_to_add, axis=0)

# Concatenate the existing array with the new rows
result_array = np.concatenate((FullDistanceMatrix, new_rows), axis=0)

new_column = np.full((result_array.shape[0], 1), number_to_fill)
new_columns = np.repeat(new_column, num_cols_to_add, axis=1)

result_array = np.concatenate((result_array, new_columns), axis=1)


print(result_array)

print([x +96 for x in Allocations])

[[   0    6    9 ... 1000 1000 1000]
 [   6    0    3 ... 1000 1000 1000]
 [   9    3    0 ... 1000 1000 1000]
 ...
 [1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]
 [1000 1000 1000 ... 1000 1000 1000]]
[141, 175, 135, 164, 169, 149, 115, 140, 112, 167, 123, 137, 98, 142, 156, 163, 152, 179, 176, 153, 165, 151, 171, 130, 185, 108, 177, 158, 119, 122, 120, 182, 99, 113, 186, 154, 147, 121, 181, 161, 127, 107, 183, 106, 109, 166, 131, 128, 143, 102, 126, 117, 139, 160, 162, 174, 172, 157, 104, 168, 118, 114, 178, 110, 124, 100, 101, 180, 150, 144, 159, 125, 145, 170, 133, 132, 116, 134, 146, 103, 184, 105, 136, 173, 111, 97, 129, 155, 138, 148, 96, 96, 96, 96, 96, 96]


In [9]:
def addTemporaryFarAwayShelves(Distances):
    #This adds the temporary shelves that are super far away from everything else to the distances matrix
    number_to_fill = 100

    # Number of rows and columns to add
    num_rows_to_add = 90
    num_cols_to_add = 90


    # Create a new row filled with the specified number
    new_row = np.full((1, FullDistanceMatrix.shape[1]), number_to_fill)

    # Repeat the new row to match the number of rows to add
    new_rows = np.repeat(new_row, num_rows_to_add, axis=0)

    # Concatenate the existing array with the new rows
    result_array = np.concatenate((FullDistanceMatrix, new_rows), axis=0)

    new_column = np.full((result_array.shape[0], 1), number_to_fill)
    new_columns = np.repeat(new_column, num_cols_to_add, axis=1)

    result_array = np.concatenate((result_array, new_columns), axis=1)

    return result_array


def slightlyGreedy_ConstructionHeuristic(Distances):
    refilledProducts = False
    tempProducts = [i for i in range(1, 91)]

    tempDistances = addTemporaryFarAwayShelves(Distances)

    normalShelves = [0]*96
    extraShelves = [x for x in range(1, 91)]

    tempAllocations = normalShelves.copy() + extraShelves.copy()

    for shelf in range(96):

        mostReduced = np.inf
        bestP = None
        
        tempShelves = tempAllocations.copy()
        baseline = q1_function(tempShelves, tempDistances)[0]

        for p in tempProducts:
            
            tempShelves = tempAllocations.copy()
            tempShelves[shelf] = p

            newobj = q1_function(tempShelves, tempDistances)

            reduction = newobj[0] - baseline

            if reduction <= mostReduced:
                bestP = p
                mostReduced = reduction

        tempAllocations[shelf] = bestP

        tempProducts.remove(bestP)

        if refilledProducts == False:
            indexOfBestP_onExtraShelf = tempAllocations[96:].index(bestP)
            del tempAllocations[indexOfBestP_onExtraShelf+96]

        if len(tempProducts) == 0:
            tempProducts = [i for i in range(1, 91)]
            refilledProducts = True

        

        print(f"Assigned product {bestP} -> shelf {shelf+1}")

    return tempAllocations

    



In [10]:
results = slightlyGreedy_ConstructionHeuristic(FullDistanceMatrix)

Assigned product 34 -> shelf 1
Assigned product 31 -> shelf 2
Assigned product 35 -> shelf 3
Assigned product 71 -> shelf 4
Assigned product 70 -> shelf 5
Assigned product 15 -> shelf 6
Assigned product 18 -> shelf 7
Assigned product 30 -> shelf 8
Assigned product 2 -> shelf 9
Assigned product 51 -> shelf 10
Assigned product 76 -> shelf 11
Assigned product 58 -> shelf 12
Assigned product 67 -> shelf 13
Assigned product 6 -> shelf 14
Assigned product 49 -> shelf 15
Assigned product 50 -> shelf 16
Assigned product 83 -> shelf 17
Assigned product 4 -> shelf 18
Assigned product 11 -> shelf 19
Assigned product 28 -> shelf 20
Assigned product 10 -> shelf 21
Assigned product 38 -> shelf 22
Assigned product 48 -> shelf 23
Assigned product 33 -> shelf 24
Assigned product 52 -> shelf 25
Assigned product 60 -> shelf 26
Assigned product 78 -> shelf 27
Assigned product 43 -> shelf 28
Assigned product 8 -> shelf 29
Assigned product 55 -> shelf 30
Assigned product 63 -> shelf 31
Assigned product 1 ->

In [111]:
greedyAllocation = results.copy()

In [11]:
results = q1_function(results, FullDistanceMatrix)
for i, order in enumerate(Orders):
    print(f"order {order} picked in dist {results[1][i]} taking route {results[4][i]}")

order [50, 30, 0, 0, 0] picked in dist 102 taking route [0, 0, 0, 0, 8, 16, 0]
order [49, 18, 76, 0, 0] picked in dist 96 taking route [0, 0, 0, 7, 11, 15, 0]
order [72, 52, 51, 41, 35] picked in dist 168 taking route [0, 25, 3, 10, 41, 83, 0]
order [50, 4, 0, 0, 0] picked in dist 114 taking route [0, 0, 0, 0, 16, 18, 0]
order [76, 19, 26, 80, 6] picked in dist 168 taking route [0, 33, 11, 14, 69, 81, 0]
order [66, 67, 51, 84, 71] picked in dist 126 taking route [0, 4, 10, 13, 40, 44, 0]
order [39, 18, 29, 83, 49] picked in dist 150 taking route [0, 53, 7, 37, 15, 17, 0]
order [88, 2, 0, 0, 0] picked in dist 168 taking route [0, 0, 0, 0, 9, 64, 0]
order [63, 19, 0, 0, 0] picked in dist 60 taking route [0, 0, 0, 0, 31, 33, 0]
order [28, 11, 77, 8, 31] picked in dist 168 taking route [0, 2, 29, 19, 20, 55, 0]
order [32, 78, 38, 31, 44] picked in dist 150 taking route [0, 2, 27, 35, 22, 48, 0]
order [63, 31, 10, 68, 30] picked in dist 162 taking route [0, 2, 31, 8, 21, 51, 0]
order [88, 7

In [13]:
print(results[0])

238062


In [ ]:
def slightlyGreedy_ConstructionHeuristic(Distances):
    refilledProducts = False
    tempProducts = [i for i in range(1, 91)]

    tempDistances = addTemporaryFarAwayShelves(Distances)

    normalShelves = [0]*96
    extraShelves = [x for x in range(1, 91)]

    tempAllocations = normalShelves.copy() + extraShelves.copy()

    for shelf in range(96):

        mostReduced = np.inf
        bestP = None
        
        tempShelves = tempAllocations.copy()
        baseline = q1_function(tempShelves, tempDistances)[0]

        for p in tempProducts:
            
            tempShelves = tempAllocations.copy()
            tempShelves[shelf] = p

            newobj = q1_function(tempShelves, tempDistances)

            reduction = newobj[0] - baseline

            if reduction <= mostReduced:
                bestP = p
                mostReduced = reduction

        tempAllocations[shelf] = bestP

        tempProducts.remove(bestP)

        if refilledProducts == False:
            indexOfBestP_onExtraShelf = tempAllocations[96:].index(bestP)
            del tempAllocations[indexOfBestP_onExtraShelf+96]

        if len(tempProducts) == 0:
            tempProducts = [i for i in range(1, 91)]
            refilledProducts = True

        

        print(f"Assigned product {bestP} -> shelf {shelf+1}")

    return tempAllocations